In [121]:
# carregando os data frames
# CNAE
schema = 'codigo INTEGER, descricao STRING'

df_cnae = spark.read.format('csv')\
               .option('sep', ';')\
               .option('encoding', 'ISO-8859-1')\
               .option('header', False)\
               .option('escape', '\'')\
               .schema(schema)\
               .load('gs://desafio-final/F.K03200$Z.D10710.CNAE.csv')

# MUNICIPIOS
schema = 'codigo INTEGER, descricao STRING'

df_municipio = spark.read.format('csv')\
               .option('sep', ';')\
               .option('encoding', 'ISO-8859-1')\
               .option('header', False)\
               .option('escape', '\'')\
               .schema(schema)\
               .load('gs://desafio-final/F.K03200$Z.D10710.MUNIC.csv')

# ESTABELECIMENTOS
schema = 'cnpj_basico INTEGER, cnpj_ordem INTEGER, cnpj_dv INTEGER, identificador_matriz INTEGER, nome_fantasia STRING, \
situacao_cadastral INTEGER, data_sit_cadastral STRING, mot_sit_cadastral INTEGER, nome_cidade_ext STRING, pais INTEGER, \
data_ini_ativ STRING, cnae_fiscal_princ INTEGER, cnae_fiscal_sec INTEGER, tipo_logradouro STRING, logradouro STRING, \
numero STRING, complemento STRING, bairro STRING, cep STRING, uf STRING, municipio INTEGER, ddd1 STRING, tel1 STRING, \
ddd2 STRING, tel2 STRING, dddfax STRING, fax STRING, correio_eletronico STRING, sit_especial STRING, data_sit DATE'

df_estab = spark.read.format('csv')\
                .option('sep', ';')\
                .option('escape', '\'')\
                .option('encoding', 'ISO-8859-1')\
                .option('header', False)\
                .schema(schema)\
                .load('gs://desafio-final/estabelecimentos/')

In [122]:
# Criando as tempViews
df_cnae.createOrReplaceTempView('CNAE')
df_municipio.createOrReplaceTempView('MUNICIPIO')
df_estab.createOrReplaceTempView('ESTAB')

In [123]:
df_estab.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_sit_cadastral: string (nullable = true)
 |-- mot_sit_cadastral: integer (nullable = true)
 |-- nome_cidade_ext: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_ini_ativ: string (nullable = true)
 |-- cnae_fiscal_princ: integer (nullable = true)
 |-- cnae_fiscal_sec: integer (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- tel1: string (nullable = true)
 

In [82]:
# Pergunta 10
spark.sql('''
       select cnae_fiscal_princ
            , count(1) qtde
         
         from ESTAB    
        
        where situacao_cadastral = 2
     
     group by cnae_fiscal_princ
     
     order by 2 desc
''').show(1)

+-----------------+------+
|cnae_fiscal_princ|  qtde|
+-----------------+------+
|          4781400|991263|
+-----------------+------+
only showing top 1 row



In [81]:
# Pergunta 11
spark.sql('''
       select to_date(data_sit_cadastral, 'yyyyMMdd'),
              data_sit_cadastral
         
         from ESTAB    
        
        where situacao_cadastral = 2    
''').show(5)

+-------------------------------------+------------------+
|to_date(data_sit_cadastral, yyyyMMdd)|data_sit_cadastral|
+-------------------------------------+------------------+
|                           2016-11-30|          20161130|
|                           2016-11-30|          20161130|
|                           2016-11-30|          20161130|
|                           2016-08-01|          20160801|
|                           2016-11-30|          20161130|
+-------------------------------------+------------------+
only showing top 5 rows



In [108]:
# Pergunta 12
spark.sql('''
       select count(cnpj_basico)
         
         from ESTAB est
         
   inner join MUNICIPIO mun
           on mun.codigo = est.municipio
         
        where UPPER(uf) = 'SP' and situacao_cadastral <> 2
''').show(10)

+------------------+
|count(cnpj_basico)|
+------------------+
|           7965268|
+------------------+



In [116]:
# Pergunta 13
spark.sql('''
       select count(1)
         
         from ESTAB est
         
   inner join MUNICIPIO mun
           on mun.codigo = est.municipio
           
   inner join CNAE cnae
           on cnae.codigo = est.cnae_fiscal_princ
         
        where UPPER(mun.descricao) = 'BELO HORIZONTE'
          and UPPER(cnae.descricao) = 'CONSULTORIA EM TECNOLOGIA DA INFORMAÇÃO'
''').show(10)

+--------+
|count(1)|
+--------+
|    2325|
+--------+



In [120]:
# Pergunta 14
spark.sql('''
       select est.nome_fantasia, est.cnae_fiscal_princ
         
         from ESTAB est
         
   inner join MUNICIPIO mun
           on mun.codigo = est.municipio
           
   inner join CNAE cnae
           on cnae.codigo = est.cnae_fiscal_princ
         
        where UPPER(mun.descricao) = 'BELO HORIZONTE'   
          and est.cnae_fiscal_princ in (8533300, 8531700, 8532500, 8542200)
          and nome_fantasia like '%IGTI%'
''').show(10)

+-------------+-----------------+
|nome_fantasia|cnae_fiscal_princ|
+-------------+-----------------+
|         IGTI|          8532500|
+-------------+-----------------+



In [141]:
# Pergunta 15
spark.sql('''
       select count(1)
              
         from ESTAB   
         
        where to_date(data_sit_cadastral, 'yyyyMMdd') >= '2020-01-01' and situacao_cadastral = 2
        
''').show(5)

+--------+
|count(1)|
+--------+
| 5744961|
+--------+

